### 1.data preprocess

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd

In [54]:
#把每个数据都添加一列，不然数据不全，用不了函数
features = pd.read_csv("/Users/mhuang/code/python/abundance/data/GSE92332/sh_rep2/features.tsv", sep='\t',header=None)
features['2']='Gene Expression'
features.to_csv("/Users/mhuang/code/python/abundance/data/GSE92332/sh_rep2/features.tsv.gz", sep='\t', index=False, header=False, compression='gzip')

adata_ctrl_rep1 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/control_rep1")
adata_ctrl_rep2 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/control_rep2")
adata_ctrl_rep3 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/control_rep3")
adata_ctrl_rep4 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/control_rep4")
adata_sh_rep1 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/sh_rep1")
adata_sh_rep2 = sc.read_10x_mtx("/Users/mhuang/code/python/abundance/data/GSE92332/sh_rep2")

adata_ctrl_rep1.obs['group']='B1'
adata_ctrl_rep2.obs['group']='B2'
adata_ctrl_rep3.obs['group']='B3'
adata_ctrl_rep4.obs['group']='B4'
adata_sh_rep1.obs['group']='B9'
adata_sh_rep2.obs['group']='B10'

adata = adata_ctrl_rep1.concatenate([adata_ctrl_rep2,adata_ctrl_rep3,adata_ctrl_rep4,
                                     adata_sh_rep1,adata_sh_rep2])
del adata.obs['batch']
adata.write('./data/GSE92332/adata_all.h5ad')

/Users/mhuang/anaconda3/lib/python3.10/site-packages/anndata/_core/anndata.py:1755: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


In [19]:
UMIcounts = pd.read_csv("/Users/mhuang/code/python/abundance/data/GSE92332/GSE92332_SalmHelm_UMIcounts.txt",sep='\t')
# FullLength_TPM = pd.read_csv("/Users/mhuang/code/python/abundance/data/GSE92332/GSE92332_SalmHelmFullLength_TPM.txt",sep='\t')
meta = UMIcounts.columns.str.split('_', expand=True)
meta = pd.DataFrame(meta.tolist(), columns=['group', 'cell.name', 'dataset.type', 'cell.type'])

In [24]:
meta.groupby("dataset.type").count()
#另外两个实验组可以拿去做别的对比实验

,group,cell.name,cell.type
dataset.type,,,
Control,3240,3240,3240
Hpoly.Day10,2711,2711,2711
Hpoly.Day3,2121,2121,2121
Salmonella,1770,1770,1770


In [233]:
import anndata as ad
def add_celltype(adata:ad.AnnData, group:list, meta:pd.DataFrame):
    '''
    处理Haber等人的数据
    '''
    cell_indeies_list = []
    celltype_meta =pd.DataFrame()
    num = 0
    
    for i in range(len(group)):
        meta_group = meta[meta['group']==group[i]]
        meta_group_list = meta_group['cell.name'].tolist()
        cell_group = adata[adata.obs['group']==group[i]].obs.index.str.split('-')
        celltype_meta_group = meta['cell.type'][meta['group']==group[i]]
        celltype_meta = pd.concat([celltype_meta, celltype_meta_group],axis=0)

        meta_index_list=[]
        cell_index_list=[]
        for j in range(len(cell_group)):
            if cell_group[j][0] in meta_group_list:
                meta_index = meta_group_list.index(cell_group[j][0])
                meta_index_list.append(meta_index)
                cell_index_list.append(j)
        
        #检查这种偷懒做法有没问题(前面的计算假定所有细胞都是按首字母排序的)
        meta_index_list2 = sorted(meta_index_list)
        cell_index_list2 = sorted(cell_index_list)
        if (meta_index_list != meta_index_list2)or(cell_index_list != cell_index_list2):
            print("==> 第 "+group[i]+" 组得重新写的复杂点.")
        else:
            if i==0:
                lst = [m for m in cell_index_list]
            else:
                lst = [m+num for m in cell_index_list]
            cell_indeies_list += lst

        num += len(cell_group)

    #把结果拼一起
    adata = adata[cell_indeies_list,:]
    celltype_meta.index = adata.obs.index
    adata.obs['cell.type']=celltype_meta
    
    return adata


In [234]:
group = ['B1','B2','B3','B4','B9','B10']
adata_1 = add_celltype(adata, group, meta)
adata_1.write('./data/GSE92332/adata_all.h5ad')

/var/folders/r9/z9c31xz10kbdyw7l1crsbwdw0000gn/T/ipykernel_31006/4275916890.py:42: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['cell.type']=celltype_meta


### 2.run_DCATS

In [235]:
adata_1

AnnData object with n_obs × n_vars = 5010 × 27775
    obs: 'group', 'cell.type'
    var: 'gene_ids', 'feature_types'

In [ ]:
def DCATS(source:ad.AnnData, target:ad.AnnData, random_state:int):
    importr("DCATS")
    globalenv['seed']=random_state
    globalenv['source'] = anndata2ri.py2rpy(source)
    globalenv['target'] = anndata2ri.py2rpy(target)
    r("""
    set.seed(seed)
    
    """)
    return

In [7]:
import anndata2ri
from rpy2.robjects import r, globalenv
from rpy2.robjects.packages import importr


# def CAMLU(train: ad.AnnData, test: ad.AnnData, random_state: int):
#     importr("CAMLU")
#     globalenv['seed'] = random_state
#     globalenv['train'] = anndata2ri.py2rpy(train)
#     globalenv['test'] = anndata2ri.py2rpy(test)
#     r("""
#     set.seed(seed)
#     label <- CAMLU(x_train = assay(train,'X'),
#                    x_test = assay(test,'X'),
#                    full_annotation = FALSE,
#                    ngene=3000, lognormalize=TRUE)
#     """)
#     label = list(r('label'))
#     return label